# OAlley API IPython example

This tutorial will show you how to use the [OAlley API](https://api.oalley.fr) to compute and display isochronous aeras.

You will need to install python packages `folium` `requests` and  `polyline`.

```
pip install folium requests polyline
```

Also, you will need an API key. Go to [api.oalley.fr](https://api.oalley.fr) and signup. It's free. Each new user receives a key with 1000 requests available.

Copy your key and paste it in the code below.

In [36]:
from IPython.display import HTML
import folium
import requests
import polyline
from urllib.parse import urlencode

baseurl = "https://api.oalley.fr/api/AppKeys/"
method = "/isochronous?"

appkey = "YOUR-API-KEY"

## Build the URL to request one zone

In this example, we prepare a request for computing a 10 min isochronous area around [Orly Airport](http://tools.wmflabs.org/geohack/geohack.php?language=fr&pagename=Orly-Sud_%28Orlyval%29&params=48.729027_N_2.369972_E_type:railwaystation)

In [32]:
point = {
    'lat': 48.729027,
    'lng': 2.369972,
    'duration' : 10 * 60
}
 
url = baseurl + appkey + method + urlencode(point)

## Execute the request

In [30]:
res = requests.get(url)
body = res.json()

if res.status_code != 200:
    print(body)

# Convert the polyline to JSON 
isozone = polyline.decode(body['polyline'])

# Trick to close the polyline, until folium library supports it
isozone.append(isozone[0])

isozone


[(48.73023, 3.03593),
 (48.85478, 3.01822),
 (48.98948, 2.98128),
 (49.02332, 2.80841),
 (49.1015, 2.74274),
 (49.16621, 2.65879),
 (49.28107, 2.59771),
 (49.177, 2.45932),
 (49.11534, 2.36905),
 (49.10887, 2.296),
 (49.08541, 2.21378),
 (49.10135, 2.11711),
 (49.104, 1.99336),
 (49.05992, 1.87371),
 (49.01187, 1.68832),
 (48.86014, 1.7134),
 (48.72984, 1.69884),
 (48.61616, 1.77544),
 (48.4725, 1.75579),
 (48.39729, 1.87378),
 (48.40805, 2.04716),
 (48.34946, 2.11586),
 (48.3073, 2.20037),
 (48.34767, 2.29252),
 (48.34275, 2.37001),
 (48.28601, 2.45966),
 (48.24081, 2.5743),
 (48.23246, 2.7017),
 (48.38199, 2.72218),
 (48.47677, 2.74301),
 (48.54337, 2.82391),
 (48.62622, 2.8868),
 (48.73023, 3.03593)]

## Display the zone in a map

The `folium` python package provides a straightforward manner to display maps and plot things in it. 

In [26]:
mymap = folium.Map(location=[46, 2], zoom_start=5)

folium.PolyLine(locations=isozone,weight=5).add_to(mymap)

mymap.fit_bounds(mymap.get_bounds())

mymap

## Compute multiple isochronous areas

It is also possible to request multiples zones from the server. The next cell shows a complete example.


In [37]:
durations = [60 * 5, 60 * 10, 60 * 20, 60 * 30, 60 * 50]

point = {
    'lat': 48.729027,
    'lng': 2.369972,
    'duration': 0.0
}
 
# Build urls    
urls = [baseurl + appkey + method + urlencode(dict(point, duration=duration)) for duration in durations]

zones = []

# Execute them one at a time
for url in urls:
    r = requests.get(url)
    body = r.json()
    isozone = polyline.decode(body['polyline'])
    # Trick to close the polyline, until folium library supports it
    isozone.append(isozone[0])
    zones.append(isozone)

# Plot the result
mymap = folium.Map(location=[46, 2], zoom_start=5)
for zone in zones:
    folium.PolyLine(locations=zone,weight=5).add_to(mymap)

mymap.fit_bounds(mymap.get_bounds())
mymap


## Next

So far, we have requested isochronous areas one at a time. 

If you need to compute a lot more isochronous, you might be interested in the [next](https://nbviewer.jupyter.org/github/overdrivr/interactive-geospatial-analysis/blob/master/Oalley_example_async.ipynb) example to speed up the process.

![isocronous areas aliens meme](./isochronous-areas.jpg)